# 1. Import Libraries 

In [2]:
from rdflib import Graph
from rdflib import URIRef, BNode, Literal
from rdflib import Namespace
from rdflib.namespace import OWL, RDF, RDFS, FOAF, XSD
from rdflib.util import guess_format

from SPARQLWrapper import SPARQLWrapper, JSON, N3

import pandas as pd
from pandas.api.types import is_numeric_dtype
pd.set_option('display.max_columns', None)

#Used from INM713 Semantic Web Technologies and Knowledge Graphs Lab 5
from isub import isub
from lookup import DBpediaLookup

import csv
import owlrl

import math

C:\Users\Abhimanyu Acharya\anaconda3\envs\diss_kg\lib\site-packages\rdflib_jsonld\__init__.py:12: DeprecationWarning: The rdflib-jsonld package has been integrated into rdflib as of rdflib==6.0.1.  Please remove rdflib-jsonld from your project's dependencies.
  DeprecationWarning,


# 2. Import Data

### 2.1 Import class attributes and save as list

In [3]:
predicates_df = pd.read_csv("C:/Dissertation/Data/AwA2-data/Animals_with_Attributes2/predicates.txt", sep = '\t',header = None, names = ['predicates'])
predicate_list = predicates_df['predicates'].tolist()
print(predicate_list)

['black', 'white', 'blue', 'brown', 'gray', 'orange', 'red', 'yellow', 'patches', 'spots', 'stripes', 'furry', 'hairless', 'toughskin', 'big', 'small', 'bulbous', 'lean', 'flippers', 'hands', 'hooves', 'pads', 'paws', 'longleg', 'longneck', 'tail', 'chewteeth', 'meatteeth', 'buckteeth', 'strainteeth', 'horns', 'claws', 'tusks', 'smelly', 'flys', 'hops', 'swims', 'tunnels', 'walks', 'fast', 'slow', 'strong', 'weak', 'muscle', 'bipedal', 'quadrapedal', 'active', 'inactive', 'nocturnal', 'hibernate', 'agility', 'fish', 'meat', 'plankton', 'vegetation', 'insects', 'forager', 'grazer', 'hunter', 'scavenger', 'skimmer', 'stalker', 'newworld', 'oldworld', 'arctic', 'coastal', 'desert', 'bush', 'plains', 'forest', 'fields', 'jungle', 'mountains', 'ocean', 'ground', 'water', 'tree', 'cave', 'fierce', 'timid', 'smart', 'group', 'solitary', 'nestspot', 'domestic']


### 2.2. Import Predicate Matrix

### 2.2.1 Binary

In [4]:
predicateMatrixBinary_df = pd.read_csv("C:/Dissertation/Data/AwA2-data/Animals_with_Attributes2/predicate-matrix-binary.txt", sep = " ",header = None, names = predicate_list)
predicateMatrixBinary_df.head()

,black,white,blue,brown,gray,orange,red,yellow,patches,spots,stripes,furry,hairless,toughskin,big,small,bulbous,lean,flippers,hands,hooves,pads,paws,longleg,longneck,tail,chewteeth,meatteeth,buckteeth,strainteeth,horns,claws,tusks,smelly,flys,hops,swims,tunnels,walks,fast,slow,strong,weak,muscle,bipedal,quadrapedal,active,inactive,nocturnal,hibernate,agility,fish,meat,plankton,vegetation,insects,forager,grazer,hunter,scavenger,skimmer,stalker,newworld,oldworld,arctic,coastal,desert,bush,plains,forest,fields,jungle,mountains,ocean,ground,water,tree,cave,fierce,timid,smart,group,solitary,nestspot,domestic
0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,1,0,0,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,0,1,0,1,1,0,0,0,1,0,0,0,1,0,1,1,0,0,0,0,1,1,0,0,0,0,1,0,1,0,1,0,1,0,0,0,0,1,0,1,0,0,0
1,1,0,0,1,0,0,0,0,0,0,0,1,0,1,1,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,1,1,1,0,1,1,1,1,1,1,1,0,1,1,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,1,0,1,0,1,0,0
2,1,1,0,0,0,0,0,0,1,1,0,0,1,1,1,0,1,1,1,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1,1,1,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1,1,0,0,0
3,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0,0,0,1,1,0,0,1,1,0,1,1,0,1,0,0,0,0,1,0,0,1,0,1,0,1,0,1,1,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,1,1,1,1,0
4,1,1,0,0,0,0,0,0,1,1,0,1,1,0,1,0,0,1,0,0,0,0,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,1,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,1,1,0,1


### 2.3 Concat Class labels and predicate matrix


In [5]:
classes_df = pd.read_csv("C:/Dissertation/Data/AwA2-data/Animals_with_Attributes2/classes.txt", sep = '\t',header = None, names = ['labels','classes'])
classes_df.head()

,labels,classes
0,1,antelope
1,2,grizzly+bear
2,3,killer+whale
3,4,beaver
4,5,dalmatian


In [6]:
classes_list = ["Antelope","Grizzly bear","Killer Whale","Beaver", "Dalmatian (dog)", "Persian cat", "Horse", "German Shepherd", 
               "Blue whale", "Siamese cat", "Skunk", "Mole (animal)", "Tiger", "Hippopotamus", "Leopard", "Moose", 
               "Spider monkey", "Humpback whale", "Elephant", "Gorilla", "Ox", "Fox", "Sheep", "Seal", "Chimpanzee", 
                "Hamster", "Squirrel", "Rhinoceros","Rabbit", "Bat", "Giraffe", "Wolf", "Chihuahua (dog)",
               "Rat", "Weasel", "Otter", "Buffalo", "Zebra", "Giant panda","Deer", "Bobcat", "Pig", "Lion", "Mouse",
                "Polar bear", "Collie", "Walrus", "Raccoon", "Cattle", "Dolphin"]

In [7]:
predicate_df = pd.concat([classes_df, predicateMatrixBinary_df], axis=1)
predicate_df['corrected_class_names'] = classes_list
predicate_df

,labels,classes,black,white,blue,brown,gray,orange,red,yellow,patches,spots,stripes,furry,hairless,toughskin,big,small,bulbous,lean,flippers,hands,hooves,pads,paws,longleg,longneck,tail,chewteeth,meatteeth,buckteeth,strainteeth,horns,claws,tusks,smelly,flys,hops,swims,tunnels,walks,fast,slow,strong,weak,muscle,bipedal,quadrapedal,active,inactive,nocturnal,hibernate,agility,fish,meat,plankton,vegetation,insects,forager,grazer,hunter,scavenger,skimmer,stalker,newworld,oldworld,arctic,coastal,desert,bush,plains,forest,fields,jungle,mountains,ocean,ground,water,tree,cave,fierce,timid,smart,group,solitary,nestspot,domestic,corrected_class_names
0,1,antelope,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,1,0,0,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,0,1,0,1,1,0,0,0,1,0,0,0,1,0,1,1,0,0,0,0,1,1,0,0,0,0,1,0,1,0,1,0,1,0,0,0,0,1,0,1,0,0,0,Antelope
1,2,grizzly+bear,1,0,0,1,0,0,0,0,0,0,0,1,0,1,1,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,1,1,1,0,1,1,1,1,1,1,1,0,1,1,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,1,0,1,0,1,0,0,Grizzly bear
2,3,killer+whale,1,1,0,0,0,0,0,0,1,1,0,0,1,1,1,0,1,1,1,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1,1,1,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1,1,0,0,0,Killer Whale
3,4,beaver,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0,0,0,1,1,0,0,1,1,0,1,1,0,1,0,0,0,0,1,0,0,1,0,1,0,1,0,1,1,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,1,1,1,1,0,Beaver
4,5,dalmatian,1,1,0,0,0,0,0,0,1,1,0,1,1,0,1,0,0,1,0,0,0,0,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,1,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,1,1,0,1,Dalmatian (dog)
5,6,persian+cat,0,1,1,0,1,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,1,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,1,1,1,0,1,0,0,1,0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,1,Persian cat
6,7,horse,1,1,0,1,1,0,0,0,1,0,0,1,0,1,1,0,0,1,0,0,1,0,0,1,1,1,1,0,1,0,0,0,0,1,0,0,0,0,1,1,0,1,0,1,0,1,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,1,1,0,0,1,Horse
7,8,german+shepherd,1,0,0,1,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,1,1,0,1,1,1,0,0,0,1,0,1,0,0,0,0,1,1,0,1,0,1,0,1,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0,1,0,1,German Shepherd
8,9,blue+whale,0,0,1,0,1,0,0,0,0,1,0,0,1,1,1,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,1,1,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,1,1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,1,1,1,0,0,Blue whale
9,10,siamese+cat,1,1,0,1,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,1,1,1,0,1,1,1,0,0,0,1,0,0,0,0,0,0,1,1,0,0,1,1,0,1,1,1,1,0,1,1,1,0,0,0,0,0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,1,0,1,0,1,Siamese cat


# 3. URI generation & RDF generation

### 3.1 Set up dictionary to hold the URI's
Adapted from N3067/INM713 Semantic Web Technologies and Knowledge Graphs Lab 5 solution

In [8]:
#Dictionary that keeps the URIs. Specially useful if accessing a remote service to get a candidate URI to avoid repeated calls
stringToURI = dict()

## 3.2 Set up empty graph and Namespace

In [9]:
#Create empty graph
g = Graph()

#Define Namespace
awa2_onto_uri = "http://www.semanticweb.org/in3067-inm713/AwA2#"
      
AwA2 = Namespace(awa2_onto_uri)
dbo = Namespace("https://dbpedia.org/ontology/")

#Bind Prefixes 
g.bind("awa2", AwA2) #cw is a newly created namespace  
g.bind("dbo", dbo) 

## 3.3 Define Code to generate Triples
Reused or Adapted from IN3067/INM713 Semantic Web Technologies and Knowledge Graphs Lab 5

### 3.3.1 Modular Mapping functions 

In [10]:
# Reused or Adapted from IN3067/INM713 Semantic Web Technologies and Knowledge Graphs Lab 5

def is_nan( x):
        return (x != x)         
            
        
def createURIForEntity(name, useExternalURI):
    stringToURI[name] = AwA2 + name.replace(" ", "_")
    
    if useExternalURI is True: #We connect to online KG
        uri = getExternalKGURI(name)
        if uri!="":
            stringToURI[name]=uri
    
    return stringToURI[name]

  
def getExternalKGURI(name):
    entities = DBpediaLookup().getKGEntities(name, 2)
    current_sim = -1
    current_uri=''
    for ent in entities:           
        isub_score = isub(name, ent.label) 
        if current_sim < isub_score:
            current_uri = ent.ident
            current_sim = isub_score    
        
    return current_uri 

    
def mappingToCreateTypeTriple(dataframe, subject_column, class_type, useExternalURI):
        
        for subject in dataframe[subject_column]:
                
            if subject.lower() in stringToURI:
                entity_uri = stringToURI[subject.lower()]
            else:
                entity_uri = createURIForEntity(subject.lower(), useExternalURI)
            
            g.add((URIRef(entity_uri), RDF.type, class_type))
            print('Example genrated URI for',subject_column,' ',URIRef(entity_uri))
            print('Example generated rdf:type triple for',subject_column,' ',URIRef(entity_uri),RDF.type, class_type)
            print('')       


def mappingToCreateLiteralTriple(subject_column, object_column, predicate, datatype):
        
        for subject, lit_value in zip(dataframe[subject_column], dataframe[object_column]):
            
            if is_nan(lit_value) or lit_value==None or lit_value=="":
                pass
            
            else:
                #Uri as already created
                entity_uri = stringToURI[subject.lower()]
                    
                #Literal
                lit = Literal(lit_value, datatype=datatype)
                
                #New triple
                g.add((URIRef(entity_uri), predicate, lit))                     
                                   
        print('Example generated data property triple for',subject_column,'and',object_column,':', URIRef(entity_uri), predicate, lit)
        print('')
                         
            
def mappingToCreateObjectTriple(dataframe, subject_column, object, predicate):
        
    for subject in dataframe[subject_column]:
            
        if is_nan(object):
            pass
            
        else:
            #Uri as already created
            subject_uri = stringToURI[subject.lower()]
            object_uri = stringToURI[object.lower()]
                    
            #New triple
            g.add((URIRef(subject_uri), predicate, URIRef(object_uri)))
        print('Example generated object property triple for',URIRef(subject_uri), predicate, URIRef(object_uri))
        print('')

### 3.4 Define URIs

### 3.4.1 Animal classes

In [11]:
stringToURI.clear() #empty dict to remove all URIs from DBPedia
mappingToCreateTypeTriple(predicate_df,'corrected_class_names', AwA2.Animal, False) #Comment to use user define URI

Example genrated URI for corrected_class_names   http://www.semanticweb.org/in3067-inm713/AwA2#antelope
Example generated rdf:type triple for corrected_class_names   http://www.semanticweb.org/in3067-inm713/AwA2#antelope http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#Animal

Example genrated URI for corrected_class_names   http://www.semanticweb.org/in3067-inm713/AwA2#grizzly_bear
Example generated rdf:type triple for corrected_class_names   http://www.semanticweb.org/in3067-inm713/AwA2#grizzly_bear http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#Animal

Example genrated URI for corrected_class_names   http://www.semanticweb.org/in3067-inm713/AwA2#killer_whale
Example generated rdf:type triple for corrected_class_names   http://www.semanticweb.org/in3067-inm713/AwA2#killer_whale http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#Animal

Example genrate

### 3.4.2 Colour classes

In [12]:
colour = ['black','white','blue','brown','grey','orange','red','yellow']
colour_df = pd.DataFrame(colour, columns = ['colour'])
mappingToCreateTypeTriple(colour_df,'colour', AwA2.Colour, False) 

Example genrated URI for colour   http://www.semanticweb.org/in3067-inm713/AwA2#black
Example generated rdf:type triple for colour   http://www.semanticweb.org/in3067-inm713/AwA2#black http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#Colour

Example genrated URI for colour   http://www.semanticweb.org/in3067-inm713/AwA2#white
Example generated rdf:type triple for colour   http://www.semanticweb.org/in3067-inm713/AwA2#white http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#Colour

Example genrated URI for colour   http://www.semanticweb.org/in3067-inm713/AwA2#blue
Example generated rdf:type triple for colour   http://www.semanticweb.org/in3067-inm713/AwA2#blue http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#Colour

Example genrated URI for colour   http://www.semanticweb.org/in3067-inm713/AwA2#brown
Example generated rdf:type triple for colour   http://w

### 3.4.3 Skin Type Classes

In [13]:
skin_type = ['patches', 'spots', 'stripes', 'toughskin']
skin_pattern_df = pd.DataFrame(skin_type, columns = ['skintype'])
mappingToCreateTypeTriple(skin_pattern_df,'skintype', AwA2.SkinType, False) 

Example genrated URI for skintype   http://www.semanticweb.org/in3067-inm713/AwA2#patches
Example generated rdf:type triple for skintype   http://www.semanticweb.org/in3067-inm713/AwA2#patches http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#SkinType

Example genrated URI for skintype   http://www.semanticweb.org/in3067-inm713/AwA2#spots
Example generated rdf:type triple for skintype   http://www.semanticweb.org/in3067-inm713/AwA2#spots http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#SkinType

Example genrated URI for skintype   http://www.semanticweb.org/in3067-inm713/AwA2#stripes
Example generated rdf:type triple for skintype   http://www.semanticweb.org/in3067-inm713/AwA2#stripes http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#SkinType

Example genrated URI for skintype   http://www.semanticweb.org/in3067-inm713/AwA2#toughskin
Example generated rdf

### 3.4.4 Fur Type Classes

In [14]:
fur_type = ['furry', 'hairless']
fur_type_df = pd.DataFrame(fur_type, columns = ['furtype'])
mappingToCreateTypeTriple(fur_type_df,'furtype', AwA2.FurType, False) 

Example genrated URI for furtype   http://www.semanticweb.org/in3067-inm713/AwA2#furry
Example generated rdf:type triple for furtype   http://www.semanticweb.org/in3067-inm713/AwA2#furry http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#FurType

Example genrated URI for furtype   http://www.semanticweb.org/in3067-inm713/AwA2#hairless
Example generated rdf:type triple for furtype   http://www.semanticweb.org/in3067-inm713/AwA2#hairless http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#FurType



### 3.4.5 Body Type Classes

In [15]:
body_type = ['big', 'small', 'bulbous', 'lean', 'muscle']
body_type_df = pd.DataFrame(body_type, columns = ['bodytype'])
mappingToCreateTypeTriple(body_type_df,'bodytype', AwA2.BodyType, False) 

Example genrated URI for bodytype   http://www.semanticweb.org/in3067-inm713/AwA2#big
Example generated rdf:type triple for bodytype   http://www.semanticweb.org/in3067-inm713/AwA2#big http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#BodyType

Example genrated URI for bodytype   http://www.semanticweb.org/in3067-inm713/AwA2#small
Example generated rdf:type triple for bodytype   http://www.semanticweb.org/in3067-inm713/AwA2#small http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#BodyType

Example genrated URI for bodytype   http://www.semanticweb.org/in3067-inm713/AwA2#bulbous
Example generated rdf:type triple for bodytype   http://www.semanticweb.org/in3067-inm713/AwA2#bulbous http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#BodyType

Example genrated URI for bodytype   http://www.semanticweb.org/in3067-inm713/AwA2#lean
Example generated rdf:type triple 

### 3.4.6 Appendage Type Classes

In [16]:
appendage_type = ['flippers', 'hands', 'hooves', 'pads', 'paws']
appendage_type_df = pd.DataFrame(appendage_type, columns = ['appendagetype'])
mappingToCreateTypeTriple(appendage_type_df,'appendagetype', AwA2.AppendageType, False) 

Example genrated URI for appendagetype   http://www.semanticweb.org/in3067-inm713/AwA2#flippers
Example generated rdf:type triple for appendagetype   http://www.semanticweb.org/in3067-inm713/AwA2#flippers http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#AppendageType

Example genrated URI for appendagetype   http://www.semanticweb.org/in3067-inm713/AwA2#hands
Example generated rdf:type triple for appendagetype   http://www.semanticweb.org/in3067-inm713/AwA2#hands http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#AppendageType

Example genrated URI for appendagetype   http://www.semanticweb.org/in3067-inm713/AwA2#hooves
Example generated rdf:type triple for appendagetype   http://www.semanticweb.org/in3067-inm713/AwA2#hooves http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#AppendageType

Example genrated URI for appendagetype   http://www.semanticweb.org/

### 3.4.7 Teeth Type Classes

In [17]:
teeth_type = ['chewteeth', 'meatteeth', 'buckteeth', 'strainteeth']
teeth_type_df = pd.DataFrame(teeth_type, columns = ['teethtype'])
mappingToCreateTypeTriple(teeth_type_df,'teethtype', AwA2.TeethType, False) 

Example genrated URI for teethtype   http://www.semanticweb.org/in3067-inm713/AwA2#chewteeth
Example generated rdf:type triple for teethtype   http://www.semanticweb.org/in3067-inm713/AwA2#chewteeth http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#TeethType

Example genrated URI for teethtype   http://www.semanticweb.org/in3067-inm713/AwA2#meatteeth
Example generated rdf:type triple for teethtype   http://www.semanticweb.org/in3067-inm713/AwA2#meatteeth http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#TeethType

Example genrated URI for teethtype   http://www.semanticweb.org/in3067-inm713/AwA2#buckteeth
Example generated rdf:type triple for teethtype   http://www.semanticweb.org/in3067-inm713/AwA2#buckteeth http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#TeethType

Example genrated URI for teethtype   http://www.semanticweb.org/in3067-inm713/AwA2#strai

### 3.4.8 Movement Type Classes

In [18]:
movement_type = ['flys', 'hops', 'swims', 'tunnels', 'walks']
movement_type_df = pd.DataFrame(movement_type, columns = ['movementtype'])
mappingToCreateTypeTriple(movement_type_df,'movementtype', AwA2.MovementType, False) 

Example genrated URI for movementtype   http://www.semanticweb.org/in3067-inm713/AwA2#flys
Example generated rdf:type triple for movementtype   http://www.semanticweb.org/in3067-inm713/AwA2#flys http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#MovementType

Example genrated URI for movementtype   http://www.semanticweb.org/in3067-inm713/AwA2#hops
Example generated rdf:type triple for movementtype   http://www.semanticweb.org/in3067-inm713/AwA2#hops http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#MovementType

Example genrated URI for movementtype   http://www.semanticweb.org/in3067-inm713/AwA2#swims
Example generated rdf:type triple for movementtype   http://www.semanticweb.org/in3067-inm713/AwA2#swims http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#MovementType

Example genrated URI for movementtype   http://www.semanticweb.org/in3067-inm713/AwA2#tun

### 3.4.9 Speed Type Classes

In [19]:
speed_type = ['fast','slow']
speed_type_df = pd.DataFrame(speed_type, columns = ['speedtype'])
mappingToCreateTypeTriple(speed_type_df,'speedtype', AwA2.MovementSpeed, False) 

Example genrated URI for speedtype   http://www.semanticweb.org/in3067-inm713/AwA2#fast
Example generated rdf:type triple for speedtype   http://www.semanticweb.org/in3067-inm713/AwA2#fast http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#MovementSpeed

Example genrated URI for speedtype   http://www.semanticweb.org/in3067-inm713/AwA2#slow
Example generated rdf:type triple for speedtype   http://www.semanticweb.org/in3067-inm713/AwA2#slow http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#MovementSpeed



### 3.4.10 Strength Type Classes

In [20]:
strength_type = ['strong','weak']
strength_type_df = pd.DataFrame(strength_type, columns = ['strengthtype'])
mappingToCreateTypeTriple(strength_type_df,'strengthtype', AwA2.StrengthType, False) 

Example genrated URI for strengthtype   http://www.semanticweb.org/in3067-inm713/AwA2#strong
Example generated rdf:type triple for strengthtype   http://www.semanticweb.org/in3067-inm713/AwA2#strong http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#StrengthType

Example genrated URI for strengthtype   http://www.semanticweb.org/in3067-inm713/AwA2#weak
Example generated rdf:type triple for strengthtype   http://www.semanticweb.org/in3067-inm713/AwA2#weak http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#StrengthType



### 3.4.11 Locomotion Type Classes

In [21]:
locomotion_type = ['bipedal', 'quadrapedal']
locomotion_type_df = pd.DataFrame(locomotion_type, columns = ['locomotiontype'])
mappingToCreateTypeTriple(locomotion_type_df,'locomotiontype', AwA2.LocomotionType, False) 

Example genrated URI for locomotiontype   http://www.semanticweb.org/in3067-inm713/AwA2#bipedal
Example generated rdf:type triple for locomotiontype   http://www.semanticweb.org/in3067-inm713/AwA2#bipedal http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#LocomotionType

Example genrated URI for locomotiontype   http://www.semanticweb.org/in3067-inm713/AwA2#quadrapedal
Example generated rdf:type triple for locomotiontype   http://www.semanticweb.org/in3067-inm713/AwA2#quadrapedal http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#LocomotionType



### 3.4.12 Activity Type Classes

In [23]:
activity_type = ['active', 'inactive']
activity_type_df = pd.DataFrame(activity_type, columns = ['activitytype'])
mappingToCreateTypeTriple(activity_type_df,'activitytype', AwA2.ActivityType, False) 

Example genrated URI for activitytype   http://www.semanticweb.org/in3067-inm713/AwA2#active
Example generated rdf:type triple for activitytype   http://www.semanticweb.org/in3067-inm713/AwA2#active http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#ActivityType

Example genrated URI for activitytype   http://www.semanticweb.org/in3067-inm713/AwA2#inactive
Example generated rdf:type triple for activitytype   http://www.semanticweb.org/in3067-inm713/AwA2#inactive http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#ActivityType



### 3.4.13 Activity Period Classes

In [24]:
activity_period = ['nocturnal']
activity_period_df = pd.DataFrame(activity_period, columns = ['activityperiod'])
mappingToCreateTypeTriple(activity_period_df,'activityperiod', AwA2.ActivityPeriod, False) 

Example genrated URI for activityperiod   http://www.semanticweb.org/in3067-inm713/AwA2#nocturnal
Example generated rdf:type triple for activityperiod   http://www.semanticweb.org/in3067-inm713/AwA2#nocturnal http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#ActivityPriod

Example genrated URI for activityperiod   http://www.semanticweb.org/in3067-inm713/AwA2#diurnal
Example generated rdf:type triple for activityperiod   http://www.semanticweb.org/in3067-inm713/AwA2#diurnal http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#ActivityPriod



### 3.4.14 Diet Type Classes

In [25]:
diet_type = ['fish', 'meat', 'plankton', 'vegetation', 'insects']
diet_type_df = pd.DataFrame(diet_type, columns = ['diettype'])
mappingToCreateTypeTriple(diet_type_df,'diettype', AwA2.DietType, False) 

Example genrated URI for diettype   http://www.semanticweb.org/in3067-inm713/AwA2#fish
Example generated rdf:type triple for diettype   http://www.semanticweb.org/in3067-inm713/AwA2#fish http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#DietType

Example genrated URI for diettype   http://www.semanticweb.org/in3067-inm713/AwA2#meat
Example generated rdf:type triple for diettype   http://www.semanticweb.org/in3067-inm713/AwA2#meat http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#DietType

Example genrated URI for diettype   http://www.semanticweb.org/in3067-inm713/AwA2#plankton
Example generated rdf:type triple for diettype   http://www.semanticweb.org/in3067-inm713/AwA2#plankton http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#DietType

Example genrated URI for diettype   http://www.semanticweb.org/in3067-inm713/AwA2#vegetation
Example generated rdf:type

### 3.4.15 Food Collection Classes

In [26]:
food_collection_type = ['forager', 'grazer', 'hunter', 'scavenger', 'skimmer', 'stalker']
food_collection_type_df = pd.DataFrame(food_collection_type, columns = ['foodcollectiontype'])
mappingToCreateTypeTriple(food_collection_type_df,'foodcollectiontype', AwA2.FoodCollectionType, False) 

Example genrated URI for foodcollectiontype   http://www.semanticweb.org/in3067-inm713/AwA2#forager
Example generated rdf:type triple for foodcollectiontype   http://www.semanticweb.org/in3067-inm713/AwA2#forager http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#FoodCollectionType

Example genrated URI for foodcollectiontype   http://www.semanticweb.org/in3067-inm713/AwA2#grazer
Example generated rdf:type triple for foodcollectiontype   http://www.semanticweb.org/in3067-inm713/AwA2#grazer http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#FoodCollectionType

Example genrated URI for foodcollectiontype   http://www.semanticweb.org/in3067-inm713/AwA2#hunter
Example generated rdf:type triple for foodcollectiontype   http://www.semanticweb.org/in3067-inm713/AwA2#hunter http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#FoodCollectionType

Example genrated URI fo

### 3.4.16 World Type Classes

In [27]:
world_type = ['new_world','old_world']
world_type_df = pd.DataFrame(world_type, columns = ['worldtype'])
mappingToCreateTypeTriple(world_type_df,'worldtype', AwA2.WorldType, False) 

Example genrated URI for worldtype   http://www.semanticweb.org/in3067-inm713/AwA2#forager
Example generated rdf:type triple for worldtype   http://www.semanticweb.org/in3067-inm713/AwA2#forager http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#WorldType

Example genrated URI for worldtype   http://www.semanticweb.org/in3067-inm713/AwA2#grazer
Example generated rdf:type triple for worldtype   http://www.semanticweb.org/in3067-inm713/AwA2#grazer http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#WorldType

Example genrated URI for worldtype   http://www.semanticweb.org/in3067-inm713/AwA2#hunter
Example generated rdf:type triple for worldtype   http://www.semanticweb.org/in3067-inm713/AwA2#hunter http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#WorldType

Example genrated URI for worldtype   http://www.semanticweb.org/in3067-inm713/AwA2#scavenger
Example gen

### 3.4.17 Habitat Type Classes

In [28]:
habitat_type = ['arctic', 'coastal', 'desert', 'bush', 'plains', 'forest', 'fields', 'jungle', 'mountains', 'ocean']
habitat_type_df = pd.DataFrame(habitat_type, columns = ['habitattype'])
mappingToCreateTypeTriple(habitat_type_df,'habitattype', AwA2.HabitatType, False) 

Example genrated URI for habitattype   http://www.semanticweb.org/in3067-inm713/AwA2#arctic
Example generated rdf:type triple for habitattype   http://www.semanticweb.org/in3067-inm713/AwA2#arctic http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#HabitatType

Example genrated URI for habitattype   http://www.semanticweb.org/in3067-inm713/AwA2#coastal
Example generated rdf:type triple for habitattype   http://www.semanticweb.org/in3067-inm713/AwA2#coastal http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#HabitatType

Example genrated URI for habitattype   http://www.semanticweb.org/in3067-inm713/AwA2#desert
Example generated rdf:type triple for habitattype   http://www.semanticweb.org/in3067-inm713/AwA2#desert http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#HabitatType

Example genrated URI for habitattype   http://www.semanticweb.org/in3067-inm713/AwA2#b

### 3.4.18 Home Type Classes

In [30]:
home_type = ['ground', 'water', 'tree', 'cave']
home_type_df = pd.DataFrame(home_type, columns = ['hometype'])
mappingToCreateTypeTriple(home_type_df,'hometype', AwA2.HomeType, False) 

Example genrated URI for hometype   http://www.semanticweb.org/in3067-inm713/AwA2#ground
Example generated rdf:type triple for hometype   http://www.semanticweb.org/in3067-inm713/AwA2#ground http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#HomeType

Example genrated URI for hometype   http://www.semanticweb.org/in3067-inm713/AwA2#water
Example generated rdf:type triple for hometype   http://www.semanticweb.org/in3067-inm713/AwA2#water http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#HomeType

Example genrated URI for hometype   http://www.semanticweb.org/in3067-inm713/AwA2#tree
Example generated rdf:type triple for hometype   http://www.semanticweb.org/in3067-inm713/AwA2#tree http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/in3067-inm713/AwA2#HomeType

Example genrated URI for hometype   http://www.semanticweb.org/in3067-inm713/AwA2#cave
Example generated rdf:type triple 

### 3.4.19 Home Type Classes

In [ ]:
temperament_type = ['ground', 'water', 'tree', 'cave']
temperament_type_df = pd.DataFrame(temperament_type, columns = ['temperamenttype'])
mappingToCreateTypeTriple(temperament_type_df,'temperamenttype', AwA2.TemperamentType, False) 


In [59]:
#mappingToCreateObjectTriple(predicate_df.loc[predicate_df['black']==1],'corrected_class_names', 'black', AwA2.hasColour)
mappingToCreateObjectTriple(predicate_df.loc[predicate_df['black']==1],'corrected_class_names', 'active', AwA2.hasActivityType)


KeyError: 'active'